In [1]:
import os
import xarray as xr
import hvplot.xarray
import pandas as pd
import panel as pn
import intake

In [2]:
Regs = ['gl', 'hn', 'tr', 'hs', 'as']
Exps = ['DTC']
Stats = ['RMSE', 'VIES', 'MEAN']

data = '20230216002023030300'

In [3]:
#catalog = intake.open_catalog('https://raw.githubusercontent.com/cfbastarz/panel_tests/main/catalog.yml')
catalog = intake.open_catalog('catalog-no_proxy.yml')

In [4]:
ds1 = catalog.scantec_gl_rmse_dtc.to_dask()

/home/carlos/miniconda3/envs/panel/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [5]:
ds1

<xarray.Dataset>
Dimensions:  (lat: 401, lon: 901, time: 12)
Coordinates:
  * lat      (lat) float32 -80.0 -79.6 -79.2 -78.8 -78.4 ... 78.8 79.2 79.6 80.0
  * lon      (lon) float32 0.0 0.4 0.8 1.2 1.6 ... 358.4 358.8 359.2 359.6 360.0
  * time     (time) datetime64[ns] 2023-02-16 2023-02-17 ... 2023-02-27
Data variables: (12/19)
    pslc000  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    temp250  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    temp500  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    temp850  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    umes500  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    umes850  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    ...       ...
    vvel250  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    vvel500  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    vvel850  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    zgeo250  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    zgeo500  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
    zgeo850  (time, lat, lon) float32 dask.array<chunksize=(1, 401, 901), meta=np.ndarray>
Attributes:
    comment:  virtual temperatura @ 925 hpa [k]
    pdef:     None
    storage:  99
    title:    
    undef:    -999.9

In [10]:
Vars = list(ds1.variables)
Vars.remove('time')
Vars.remove('lat')
Vars.remove('lon')

variable_list = Vars
variable = pn.widgets.Select(name='Variable', value=variable_list[0], options=variable_list)

region = pn.widgets.Select(name='Region', value=Regs[0], options=Regs)
experiment = pn.widgets.Select(name='Experiment', value=Exps[0], options=Exps)
statistic = pn.widgets.Select(name='Statistic', value=Stats[0], options=Stats)

test_list = ['ref_GFS', 'ref_Era5', 'ref_PAnl']
test = pn.widgets.Select(name='Reference', value=test_list[0], options=test_list)

@pn.cache
def loadData(lfname):
    return catalog[lfname].to_dask()

@pn.depends(variable, region, experiment, statistic, test)
def plotFields(variable, region, experiment, statistic, test):
    if test == 'ref_GFS': ttest = 'T1'
    if test == 'ref_Era5': ttest = 'T2'
    if test == 'ref_PAnl': ttest = 'T3'
    lfname = 'scantec_' + region.lower() + '_' + statistic.lower() + '_' + experiment.lower()
    #dfs = catalog[lfname].to_dask()
    dfs = loadData(lfname)
    cmin=dfs[variable].min()
    cmax=dfs[variable].max()
    #cmap='tab20c_r'
    if region == 'as': 
        frame_width=500
    else: 
        frame_width=960
    ax = dfs[variable].hvplot(groupby='time', clim=(cmin, cmax), widget_type='scrubber', widget_location='bottom', 
                              frame_width=frame_width)#, cmap=cmap)
    return pn.Column(ax, sizing_mode='stretch_width')

@pn.depends(variable, region, experiment, statistic, test)
def myName(variable, region, experiment, statistic, test):
    if test == 'ref_GFS': ttest = 'T1'
    if test == 'ref_Era5': ttest = 'T2'
    if test == 'ref_PAnl': ttest = 'T3'
    lfname = 'scantec_' + region.lower() + '_' + statistic.lower() + '_' + experiment.lower()

    return lfname

card_parameters = pn.Card(variable, region, experiment, statistic, test, title='Parameters', collapsed=False)

settings = pn.Column(card_parameters)

pn.template.FastListTemplate(
    site="My Dashboard", title="panel_tests", sidebar=[settings],
    main=["My test.", myName, plotFields], 
).show();
#).servable();

Launching server at http://localhost:45239


/home/carlos/miniconda3/envs/panel/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
[0215/093423.260218:WARNING:chrome_main_linux.cc(80)] Read channel stable from /app/extra/CHROME_VERSION_EXTRA


[0215/093423.373883:WARNING:chrome_main_linux.cc(80)] Read channel stable from /app/extra/CHROME_VERSION_EXTRA
Opening in existing browser session.


/home/carlos/miniconda3/envs/panel/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
